# Exploring Airports in Australia

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import geoapify_key

## DataFrame Preparation

In [4]:
# Import cities file as DataFrame
airports = pd.read_csv('Cities.csv')
airports

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide
5,Gold Coast
6,Townsville
7,Williamtown


In [10]:
# Add columns the airports data we will fetch using the Geoapify API
# Note that we used "" to specify initial entry.
airports['Lat'] = ''
airports['Lon'] = ''
airports['Airport Name'] = ''
airports['IATA Name'] = ''
airports['Airport Address'] = ''
airports['Distance'] = ''
airports['Website'] = ''
airports

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,
5,Gold Coast,,,,,,,
6,Townsville,,,,,,,
7,Williamtown,,,,,,,


## Look for Geographical Coordinates for Each City

In [11]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [20]:
# Print a message to follow up the airport search
print("Starting airport search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in airports.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    name = airports.loc[index, 'City']
    place = f"{name}, Austrailia"

    # Add the current city to the parameters
    params['name'] = place

    # Make the API request
    airport_data = requests.get(base_url, params=params)
    
    # Convert response to JSON
    airport_data = airport_data.json()

    # Extract latitude and longitude
    try:
        airports.loc[index, "Lat"] = airport_data['results'][0]['lat']
        airports.loc[index, "Lon"] = airport_data['results'][0]['lon']
    except (KeyError, IndexError):
        pass


Starting airport search


In [22]:
airports

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,40.737415,-73.861607,,,,,
1,Melbourne,28.110702,-80.637624,,,,,
2,Perth,43.01757,-74.19402,,,,,
3,Cairns,45.526965,-122.675688,,,,,
4,Adelaide,-34.921,138.596767,,,,,
5,Gold Coast,42.590757,-88.435526,,,,,
6,Townsville,-19.269225,146.807817,,,,,
7,Williamtown,-32.800258,151.84868,,,,,


## Retrieve Airports' Information

In [44]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for index, row in airports.iterrows():
    
    # Get the city's name
    name = airports.loc[index, 'City']
    
    # Get latitude, longitude from the DataFrame
    latitude = airports.loc[index, "Lat"]
    longitude = airports.loc[index, 'Lon']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
           
    # Make an API request using the params dictionary
    closet_airports = requests.get(base_url, params = params)
        
    # Convert the API response to JSON format
    closet_airports = closet_airports.json()
    
    # Grab the first airport from the results and store the details in the DataFrame
    try:
        airports.loc[index, "Airport Name"] = closet_airports['features'][0]['properties']['name']
        airports.loc[index, "IATA Name"] = closet_airports['features'][0]['properties']['datasource']['raw']['iata']
        airports.loc[index, 'Airport Address'] = closet_airports['features'][0]['properties']['address_line2']
        airports.loc[index, 'Distance'] = closet_airports['features'][0]['properties']['distance']
        airports.loc[index, "Website"] = closet_airports['features'][0]['properties']['datasource']['raw']['website']
    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {airports.loc[index, 'Airport Name']}")
        


# Display sample data


Starting airport details search
distance not found for Orlando Melbourne International Airport
iata not found for Snow Field
website not found for Portland International Airport
iata not found for Burlington Municipal Airport


In [45]:
airports                                             

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,40.737415,-73.861607,LaGuardia Airport,LGA,"Flushing Bay Promenade, New York, NY 11371, Un...",3176,https://www.laguardiaairport.com/
1,Melbourne,28.110702,-80.637624,Orlando Melbourne International Airport,MLB,"Wright Court, Melbourne, FL 32935, United Stat...",,
2,Perth,43.01757,-74.19402,Snow Field,,,,
3,Cairns,45.526965,-122.675688,Portland International Airport,PDX,"Northeast Argyle Street, Portland, OR 97211, U...",7246,
4,Adelaide,-34.921,138.596767,Adelaide Airport,ADL,"Streeters Road, North Plympton SA 5037, Australia",4828,https://www.adelaideairport.com.au/
5,Gold Coast,42.590757,-88.435526,Burlington Municipal Airport,,,,
6,Townsville,-19.269225,146.807817,Townsville Airport,TSV,"John Melton Black Drive, Garbutt QLD 4810, Aus...",3579,https://www.townsvilleairport.com.au/
7,Williamtown,-32.800258,151.84868,Newcastle Airport,NTL,"55 Slades Road, Williamtown NSW 2318, Australia",703,https://www.newcastleairport.com.au
